In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                 # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../")            #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
using GraphPlot
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff

# Tools

## Enumerator

Tools to compute a running enumeration of a "list" of objects.

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

## Running Statistics

Tools to compute running statistics such as mean and std from a stream of numbers.

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

In [ ]:
list_v = 10 .+ 5.0*randn(1000)
;

In [ ]:
rs = RunningStat()

In [ ]:
for v in list_v
    push!(rs,v)
end

In [ ]:
mean(rs),std(rs)

In [ ]:
mean(list_v),std(list_v)

## Coefficient of determiantion, $R^2$

Tools to compute the $R^2$ coefficient to evaluate the fit of functions.

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

In [ ]:
# Define the model function
@. model(x, p) = p[1] * x + p[2]

# Sample data (example)
x_data = [1.0, 2.0, 3.0, 4.0, 5.0]
y_data = [2.2, 2.8, 3.6, 4.5, 5.1]

# Initial guess for parameters
p0 = [1.0, 1.0]

# Perform curve fitting
fit = LsqFit.curve_fit(model, x_data, y_data, p0)

# Compute fitted values
y_fit = model(x_data, coef(fit))

# Compute R_sqr
R_sqr = R_squared(y_data,y_fit)

In [ ]:
plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"x")
plot!(ylabel=L"y")
scatter!(x_data,y_data,label="data")
plot!(x_data,y_fit,label="fit, \$R^2=0.995\$")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
#plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(
#    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
#    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
#    tickfont=font(11),
#)

## Sigmoid functions

The logistic sigmoid function is

$$
f(x) = \frac{A}{1+e^{-B(x-x_0)}}
$$

Its derivative is

$$
f'(x) = \frac{A\,B\,e^{-B(x-x_0)}}{(1+e^{-B(x-x_0)})^2}
$$

and its primitive is

$$
F(x) = \int dx\,f(x) = \frac{A}{B}\ln(1+e^{B(x-x_0)})+C
$$


In [ ]:
x0 = 1.0
A = 1.0
B = 3.0
C = 0.0
p0 = [x0,A,B,C] # p1=x0, p2=A, p3=B, p4=C
@. sigmoid(x,p)  = p[2]/(1+exp(-p[3]*(x-p[1]))) # sigmoid
@. dsigmoid(x,p) = p[2]*p[3]*exp(-p[3]*(x-p[1]))/(1+exp(-p[3]*(x-p[1])))^2 # derivative of sigmoid
@. psigmoid(x,p) = p[2]/p[3]*log(1+exp(p[3]*(x-p[1])))+p[4] # primitive of sigmoid

### Test

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"f(x)",
)
plot!(-2:0.01:4.0,x->sigmoid(x,p0),label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"df/dx",    
)
plot!(-2:0.01:4.0,x->dsigmoid(x,p0),label="")
scatter!([x0],[A*B/4],label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"F(x)",
)
plot!(-2:0.01:4.0,x->psigmoid(x,p0),label="")

## Log-range

Tools to create a range of numbers but in logarithmic scale.

In [ ]:
function logrange(start,stepmul,stop)
    @assert start>0
    @assert stepmul>1
    r = []
    s = start
    while s <= stop
        push!(r,s)
        s *= stepmul
    end
    return r
end

### Test

In [ ]:
logrange(1e-4,1.1,1)

# Generate networks

In [ ]:
function main_component(g)
    c = connected_components(g)
    _,i = findmax(length.(c))
    return g[c[i]]
end

In [ ]:
sorted_connected_components(g) = sort(connected_components(g), by = c -> length(c),rev=true)

In [ ]:
"""
Generates a 1D lattice Graph of N nodes and k=2z nearest neighbors.
"""
function lattice_1D_to_g(N,z=1)
    g = Graph(N)
    for i in 1:N
        for j in 1:z
            if i+j <= N
                add_edge!(g,i,i+j)            
            end
        end
    end
    return g
end

## Tests

In [ ]:
sorted_connected_components(g)

# Load networks from raw data and save them in JLD2 format

In [ ]:
list_nets_dat = """../redes/Italy/red_crossing.dat
../redes/Italy/red_counterattack.dat
../redes/Italy/red_pressure_loss.dat
../redes/Italy/red_T_build_up.dat
../redes/Italy/red_direct_play.dat
../redes/Italy/red_pressure_point.dat
../redes/Italy/red_shots.dat
../redes/Italy/red_flow_rate.dat
../redes/Italy/red_T_maintenance.dat
../redes/Italy/red_T_zona_media.dat
../redes/Germany/red_crossing.dat
../redes/Germany/red_counterattack.dat
../redes/Germany/red_pressure_loss.dat
../redes/Germany/red_T_build_up.dat
../redes/Germany/red_direct_play.dat
../redes/Germany/red_pressure_point.dat
../redes/Germany/red_shots.dat
../redes/Germany/red_flow_rate.dat
../redes/Germany/red_T_maintenance.dat
../redes/Germany/red_T_zona_media.dat
../redes/Spain/red_crossing.dat
../redes/Spain/red_counterattack.dat
../redes/Spain/red_pressure_loss.dat
../redes/Spain/red_T_build_up.dat
../redes/Spain/red_direct_play.dat
../redes/Spain/red_pressure_point.dat
../redes/Spain/red_shots.dat
../redes/Spain/red_flow_rate.dat
../redes/Spain/red_T_maintenance.dat
../redes/Spain/red_T_zona_media.dat
../redes/England/red_crossing.dat
../redes/England/red_counterattack.dat
../redes/England/red_pressure_loss.dat
../redes/England/red_T_build_up.dat
../redes/England/red_direct_play.dat
../redes/England/red_pressure_point.dat
../redes/England/red_shots.dat
../redes/England/red_flow_rate.dat
../redes/England/red_T_maintenance.dat
../redes/England/red_T_zona_media.dat
../redes/France/red_crossing.dat
../redes/France/red_counterattack.dat
../redes/France/red_pressure_loss.dat
../redes/France/red_T_build_up.dat
../redes/France/red_direct_play.dat
../redes/France/red_pressure_point.dat
../redes/France/red_shots.dat
../redes/France/red_flow_rate.dat
../redes/France/red_T_maintenance.dat
../redes/France/red_T_zona_media.dat"""

In [ ]:
redes_jld2 = Dict()
for archivo in split(list_nets_dat)
    println(archivo)
    vec_i = Vector{Int64}()
    vec_j = Vector{Int64}()
    vec_w = Vector{Float64}()    
    open(archivo) do fh
        for line in readlines(fh)[2:end]
            cols=split(replace(line,"," => " "))
            #println(cols)
            push!(vec_i,parse(Int64,cols[1]))
            push!(vec_j,parse(Int64,cols[2]))
            push!(vec_w,parse(Float64,cols[3]))
        end
    end
    tag = replace(archivo, "../redes/" => "")
    tag = replace(tag, ".dat" => "")
    tag = replace(tag, "red_" => "")
    tag = replace(tag, "/" => " ")
    tag = split(tag)
    tag = (tag[1],tag[2])
    redes_jld2[tag] = (vec_i,vec_j,vec_w)
end

In [ ]:
redes_jld2

In [ ]:
@save "../redes/redes_dict.jld2" redes_jld2

In [ ]:
@load "../redes/redes_dict.jld2" redes_jld2

In [ ]:
redes_jld2

In [ ]:
redes_jld2[("France","crossing")]

# References

[1] ...

In [ ]:
g = erdos_renyi(6, 0.4)
g = main_component(g)

In [ ]:
I_3c = g_to_3_cliques(g) # We use I123 instead of I since in Julia I is a reserved name for the identity matrix.

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
sc.K[1]

In [ ]:
sc.I[1]

In [ ]:
sc.K[2]

In [ ]:
sc.I[2]

In [ ]:
sc.alpha[1]

In [ ]:
sc.alpha[2]

In [ ]:
nnz_alpha(sc)

The $k$-clique complex of $g$ is the simplicial complex obtained by considering all the $q$-cliques of $g$ for $q\leq k$.

First, the 3-clique complex of the network of Fig. (1) in [2] is created.

In [ ]:
I0 = Dict{Any,Int}()
I1 = Dict{Any,Int}()
I2 = Dict{Any,Int}()

for i in 1:8
    enum!(I0,[i])
end
I0

enum!(I1,[1,2])
enum!(I1,[1,8])
enum!(I1,[2,3])
enum!(I1,[2,6])
enum!(I1,[3,4])
enum!(I1,[3,5])
enum!(I1,[3,6])
enum!(I1,[3,8])
enum!(I1,[4,5])
enum!(I1,[4,6])
enum!(I1,[5,6])
enum!(I1,[6,7])
enum!(I1,[7,8])
I1

enum!(I2,[2,3,6])
enum!(I2,[3,4,5])
enum!(I2,[3,4,6])
enum!(I2,[3,5,6])
enum!(I2,[4,5,6])
I2

In [ ]:
I_vs_k = 

sc = I_to_SC(I_vs_k)
sc.alpha[2]

In [ ]:
#w = zeros(dim_Ck(sc,0))
f = zeros(m_k(sc,1))

I1 = sc.I[1]

f[I1[[1,2]]] = -1   # 1
f[I1[[1,8]]] = 4.2  # 2
f[I1[[2,3]]] = -2   # 3
f[I1[[2,6]]] = 8.1  # 4
f[I1[[3,4]]] = 3.1  # 5
f[I1[[3,5]]] = 5.9  # 6
f[I1[[3,6]]] = 9.8  # 7
f[I1[[3,8]]] = 7.1  # 8
f[I1[[4,5]]] = 3.1  # 9
f[I1[[4,6]]] = 6.9  # 10
f[I1[[5,6]]] = 4.1  # 11
f[I1[[6,7]]] = -1   # 12
f[I1[[7,8]]] = -2   # 13

f

In [ ]:
alpha_1 = sc.alpha[1]

In [ ]:
alpha_1'

Here, $z:=d_0^*(f)$ should equal the r.h.s. of Eq. (18) in [2].

In [ ]:
z = alpha_1'*f

Here, $L_0$ should equal the matrix at the l.h.s. of Eq. (18) in [2].

In [ ]:
L0 = alpha_1'*alpha_1

In [ ]:
w = L0 \ z

Next vector should be equal to the vector $\alpha$ reported in Table 5 of [2].

In [ ]:
w .- w[1]

Next vector should equal the r.h.s. of Eq. (18) in [2].

In [ ]:
L0*w

Check that vector $g=d_0(w)=\alpha_1 w$ equals the vector $w_g$ of Table 4 in [2].

In [ ]:
g = alpha_1*w

Check equality up to numerical tolerance that $d_1(f)=d_1(g)$.

In [ ]:
norm( alpha_1'*f - alpha_1'*g )

In [ ]:
alpha_2 = sc.alpha[2]

Check the form of $L_2^{\downarrow}=\alpha_2\alpha_2^T$. It should equal the matrix at the l.h.s. of Eq. (20) in [2].

In [ ]:
L2d = alpha_2 * alpha_2'

Here, $y:=d_1(f)$ should equal the r.h.s. of Eq. (20) in [2].

In [ ]:
y = alpha_2 * f

In [ ]:
# See 
#    https://stackoverflow.com/questions/38486697/solve-a-particular-linear-system-efficiently-in-julia
#    https://iterativesolvers.julialinearalgebra.org/dev/
#    https://jso.dev/Krylov.jl/dev/solvers/ls/
# on different alternatives about how to solve the linear systems in consideration.

#u = L2d \ y
#u = pinv(Symmetric(Matrix(L2d)))*y
#u = IterativeSolvers.cg(L2d,y)     # this is faster than lsmr in the example, according to @btime
#u = IterativeSolvers.minres(L2d,y)
#u = IterativeSolvers.lsqr(L2d,y)
#u = IterativeSolvers.lsmr(L2d,y)
#u = Krylov.cg(L2d,y)[1]    
#u = Krylov.lslq(L2d,y)[1]
#u = Krylov.lsqr(L2d,y)[1]
u = Krylov.minres_qlp(L2d,y)[1]

The vector $u$ obtained is different from the vector $\gamma$ reported in Table (5) of [2], which is the following one:

In [ ]:
u_johnson = [-0.08 -1.1309 1.2359 -1.1759 1.2809]'

Is the vector $u_{\mathrm{johnson}}$ (i.e. $\gamma$) reported in [2] also a solution of Eq. (20) of [2]? We check:

In [ ]:
norm(L2d*u_johnson - y)

It actually is! What about our solution? Let us check:

In [ ]:
norm(L2d*u - y)

It is as well! Hence, there are multiple solutions, so $L_2^{\downarrow}(u)=y$ is a degenerate (i.e. underdetermined) linear system. 

Only one of the solutions of degenerate linear systems of a positive semidefinite linear operator has minimum norm $|x|$. Therefore, we ask ourselves which of the solutions to $L_2^{\downarrow}u=y$ we have found, has the smallest norm? Let us see:

In [ ]:
norm(u)

In [ ]:
norm(u_johnson)

So our solution has smaller norm than that of [2]. Hence, it is a better candidate for the unique solution of minimum norm.

Next, compute $s=d_1^*(u)=\alpha_2^T u$.

In [ ]:
s = alpha_2'*u

Check the "equality" of $\alpha_2 s = d_1(s) = d_1(f) = \alpha_2 f$.

In [ ]:
norm( alpha_2*s - alpha_2*f )

Compute $h$.

In [ ]:
h = f-s-g

In [ ]:
L1u = alpha_2'*alpha_2
L1d = alpha_1*alpha_1'
L1 = L1u + L1d

Finally check that $L_1(h)=0$.

In [ ]:
norm(L1*h)

In [ ]:
g = erdos_renyi(10, 0.4)

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
for k in 0:kappa(sc)
   println("m_$k=$(m_k(sc,k))")
end

Next, we create a 1-chain $f$ of components $f_{ij}=w_i-w_j$ where $w$ is the 0-chain of components $w_i=i$.

In [ ]:
m0=m_k(sc,0)
m1=m_k(sc,1)
w = collect(1:m0)
f = zeros(m1)
K1 = sc.K[1]
for l in 1:m1
    i,j = K1[l,:]
    f[l] = w[i]-w[j]
end

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,w,0)

In [ ]:
hd.g

In [ ]:
hd.w

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w .- minimum(hd.w)

In [ ]:
u = collect(1:m_k(sc,2))

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,u,2)

In [ ]:
hd.s

In [ ]:
"""
Generator of artificial ratings.
    agents : iterable over the agents.
    noise : Gaussian additive noise
    scale : sepparation scale between ratings
"""
function gen_artificial_ratings(agents,noise=0.0,scale=1.0)
    #@show agents
    N = length(agents)
    w = collect(agents)+noise*randn(N)
    w = ratings(N .- w)
    return scale*w
end

In [ ]:
"""
Takes a simplicial complex and a vector of ratings and generates the 1-cochain

    f_{ij} = w_i - w_j + noise*randn()
"""
function gen_artificial_f(sc,w,noise=0.0)
    m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    @assert length(w) == m_0
    f = zeros(m_1)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = w[j]-w[i] + noise*randn()
    end
    return f
end

In [ ]:
"""
Takes a vector and shifts the values of its components by a constant to make all of them non-negative.
"""
function ratings(w)
    return w .- minimum(w)
end

In [ ]:
"""
Normalizes a vector of ratings to have values between 0 and 1 inclusive.
"""
function normalized_ratings(w)
    minw = minimum(w)
    #maxw = maximum(w)
    #return (w .- minw)/(maxw-minw)
    return (w .- minw)/length(w)
end

In [ ]:
"""
Takes a vector of ratings and computes the corresponding ranking
"""
function ranking(w)
    return sortperm(w,rev=true)
end

In [ ]:
"""
Given two vectors v and u, this computes the absolute differences

    x_i = |v_i-u_i|

And returns the vector w of cumulative diferences

    w_i = sum_{j <= i} x_j

This can be used to compare two vectors of ratings or rankings
"""
function compute_cumulative_abs_diff(v,u)
    n = length(v)
    @assert length(u) == n
    cumulative_abs_diff = zeros(n)
    sum = 0
    for i in 1:n
         sum += abs.(v[i]-u[i])
         cumulative_abs_diff[i] = sum
    end
    return cumulative_abs_diff
end

In [ ]:
g = erdos_renyi(6, 0.4)

In [ ]:
w = gen_artificial_ratings(1:nv(g))

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
ratings(w .- 5)

In [ ]:
ranking(w)

In [ ]:
normalized_ratings(w)

In [ ]:
palette_colors = palette(:auto)
col1 = palette_colors[1]
col2 = palette_colors[2]
col3 = palette_colors[3]
col4 = palette_colors[4]
col5 = palette_colors[5]
col6 = palette_colors[6]
col7 = palette_colors[7]
col8 = palette_colors[8]
cols = [col1,col2,col3,col4,col5,col6,col7,col8]

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,6)
cols1 = [col1,col2]
colors_N = palette(cols1,6)

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,8)
#colors_z = palette(:hawaii,13)
cols2 = [col4,col6]
colors_z = palette(cols2,13)
colors_k = colors_z
colors_q = colors_z
colors_p = colors_z

In [ ]:
N = 10000
mean_k = 3.0
p = mean_k/(N-1)
@show N
@show mean_k
@show p
#g = erdos_renyi(6, 0.4)
g = erdos_renyi(N,p)
@show nv(g)
@show ne(g)
@show 2*ne(g)/N
#@show g
;

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="ER")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="ER")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

In [ ]:
sum(abs.(ranking(w)-ranking(hd.w)))

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="BA")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="BA")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

In [ ]:
list_cc_mean[1]/list_cc_mean[1]